In [ ]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist_m as mnistm
from data import mnist
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from data.tasks import load_task
from experiments.training import *
from util.misc import *
from util.batch import *

# Hyper-parameters
task = 4
seed = 69105
delta=0.05    
binary=True
architecture="lenet"
epsilons=[0.01]
alphas=[0.3]

project_folder = "/cephyr/users/adambre/Alvis/mnist_transfer/"
#project_folder="/cephyr/NOBACKUP/groups/snic2021-23-538/mnist_transfer/"

In [ ]:
project = 'SNIC2021-7-83'
username = 'adambre'
job = 'batch_train_single.sbatch'

In [ ]:
print('Iterating over experiments...\n')
fids = []

os.makedirs(project_folder+'logs', exist_ok=True)


np.random.seed(seed)
# for s in range(10):
#     seed+=1
#     print(seed)
for alpha in alphas:
    print("alpha:"+str(alpha))
    for epsilon in epsilons:
        print("  epsilon:"+str(epsilon)) 
        fid = project_folder+'logs/batch_t-%d_r-%d_a-%.4f_e-%.4f_d-%.4f_b-%d_A-%s' % \
            (task, seed, alpha, epsilon, delta, binary ,architecture)

        exp = 'task=%d,seed=%d,alpha=%.4f,epsilon=%.4f,delta=%.4f,'% (task, seed, alpha, epsilon, delta)\
                +'binary=%d,architecture=%s' % (binary, architecture)
        
        output = !sbatch -o "$fid"-%j.out -e "$fid"-%j.err -A $project --export="$exp" $job

        jobid = int(output[0].split(' ')[-1])
        fid = fid+'-%s' % jobid

        fids.append(fid)

In [ ]:
!squeue -u $username



In [ ]:
for fid in fids: 
    print('------')
    print(fid)
    !tail -n 10 "$fid".out
    print('\n------')
    !tail -n 5 "$fid".err
    print(' \n\n')